In [1]:
!pip install tensorflow keras

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings("ignore")

In [9]:
import tensorflow as tf
import keras
from keras import layers

print(tf.__version__)
print(keras.__version__)

2024-03-15 07:07:02.995300: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 07:07:03.000821: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 07:07:03.076254: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 07:07:04.321073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.16.1
3.0.5


In [10]:
## Importing the dataset

df = pd.read_csv("wine.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,bad
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,bad
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,bad
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,good
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,bad


## EDA

In [11]:
df.shape

(1599, 12)

In [12]:
# Check null values

df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [13]:
# Info about the dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   object 
dtypes: float64(11), object(1)
memory usage: 150.0+ KB


In [14]:
# Description

df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free sulfur dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total sulfur dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


In [15]:
# Categorical variables identification

for col in df.columns:
    if df[col].nunique() < 15 and df[col].dtypes == 'object':
        print(col, end = " ")

quality 

In [16]:
## Encoding the variables

mapping = {"good" : 1, "bad" : 0}

df["quality"] = df.quality.map(mapping)

In [17]:
df.quality.value_counts() # almost equal proportions no class imbalance

1    855
0    744
Name: quality, dtype: int64

In [18]:
X = df.drop("quality", axis = 1)
y = df.quality.values

In [19]:
## train test split
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, shuffle=True, random_state=42, test_size=0.2)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, shuffle=True, random_state=0, test_size=0.5)

print(f"X_train : {X_train.shape}, y_train : {y_train.shape}")
print(f"X_val : {X_val.shape},    y_val : {y_val.shape}")
print(f"X_test : {X_test.shape},   y_test : {y_test.shape}")

X_train : (1279, 11), y_train : (1279,)
X_val : (160, 11),    y_val : (160,)
X_test : (160, 11),   y_test : (160,)


In [20]:
## Feature scaling
from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [24]:
## Creating the model

model = keras.Sequential([
            layers.Dense(128, activation="relu", name = "input_layer"),
            layers.Dense(64, activation="relu", name = "hidden_layer_1"),
            layers.BatchNormalization(),
            layers.Dense(32, activation="relu", name = "hidden_layer_2"),
            layers.Dense(1, activation="sigmoid", name = "output_layer")
])

OPTIMIZER = "Adam"
LOSS = "binary_crossentropy"
METRICS = ["accuracy"]

checkpoint_filepath = "./model/checkpoint.model.keras"
log_dirs = "./logs"

os.makedirs(log_dirs, exist_ok=True)

my_callbacks = [
    keras.callbacks.EarlyStopping(patience=10),
    keras.callbacks.ModelCheckpoint(checkpoint_filepath),
    keras.callbacks.TensorBoard(log_dir=log_dirs, histogram_freq=1),
]

model.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = METRICS)

In [25]:
history = model.fit(X_train_scaled, y_train, 
                    batch_size = 64, 
                    epochs = 30, 
                    callbacks = my_callbacks, 
                    validation_data= (X_val, y_val), 
                    verbose = 1)

Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6245 - loss: 0.6419 - val_accuracy: 0.5125 - val_loss: 1.4751
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7986 - loss: 0.4589 - val_accuracy: 0.4875 - val_loss: 3.1608
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8082 - loss: 0.4436 - val_accuracy: 0.4938 - val_loss: 4.6270
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7974 - loss: 0.4263 - val_accuracy: 0.4938 - val_loss: 5.1642
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8179 - loss: 0.4121 - val_accuracy: 0.4750 - val_loss: 6.8368
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8294 - loss: 0.3884 - val_accuracy: 0.4750 - val_loss: 7.4506
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8357 - loss: 0.3842 - val_accuracy: 0.4750 - val_loss: 9.4233
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8535 - loss: 0.3428 - val_accuracy: 0.4750 - v

In [26]:
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (Dense)             │ (None, 128)            │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_1 (Dense)          │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,229 (141.52 KB)

 Trainable params: 12,033 (47.00 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 24,068 (94.02 KB)

None


In [27]:
print(f"Params : {model.get_weights()}")

Params : [array([[-0.01065898,  0.22596638, -0.04328841, ...,  0.06703929,
         0.2022897 ,  0.05881835],
       [ 0.12986283,  0.1542811 ,  0.11271688, ...,  0.0651516 ,
         0.04079777,  0.05135823],
       [-0.00945504,  0.04174123, -0.079173  , ..., -0.1590851 ,
        -0.18795899,  0.10115994],
       ...,
       [ 0.12181287, -0.19877385, -0.03065808, ...,  0.11011295,
        -0.22072433,  0.01776065],
       [-0.06472871, -0.00632626, -0.09012218, ..., -0.16857274,
        -0.0624286 , -0.08830173],
       [ 0.09936115, -0.07415241,  0.08467939, ...,  0.11867953,
         0.12628905,  0.04947606]], dtype=float32), array([-0.02396321,  0.00214891, -0.01296287,  0.01993941, -0.01958255,
        0.00128772,  0.00768061,  0.0217655 ,  0.01214214, -0.02305565,
       -0.02394567,  0.00328961,  0.02198179,  0.02088718,  0.0141189 ,
       -0.01088506,  0.04191541, -0.01076715, -0.02137228,  0.01856514,
       -0.02006529,  0.01905001, -0.02549089, -0.01161786,  0.00501957,
 

In [28]:
history = pd.DataFrame(history.history)
history

,accuracy,loss,val_accuracy,val_loss
0,0.695074,0.580840,0.51250,1.475075
1,0.781861,0.475080,0.48750,3.160833
2,0.802971,0.446070,0.49375,4.626954
3,0.807662,0.423788,0.49375,5.164228
4,0.825645,0.408139,0.47500,6.836756
5,0.839719,0.390206,0.47500,7.450636
6,0.844410,0.376079,0.47500,9.423305
7,0.844410,0.364260,0.47500,9.893706
8,0.853792,0.349834,0.47500,10.953220
9,0.865520,0.336411,0.48125,14.413196


In [29]:
## Evaluation
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

print(f"Test loss - {test_loss}")
print(f"Test Accuracy - {test_accuracy}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7265 - loss: 0.5165 
Test loss - 0.4991900324821472
Test Accuracy - 0.7562500238418579
